<a href="https://colab.research.google.com/github/HaiwenGuan/MLP-MNIST/blob/main/MLP_starter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Imports**

In [66]:
import os
import numpy as np
import time
import tensorflow as tf
from keras.datasets import mnist

np.random.seed(4208)
tf.random.set_seed(4208)

In [89]:
tf.executing_eagerly()
tf.__version__

'2.7.0'

In [90]:
tf.config.list_physical_devices('GPU')

[]

Generate random data

In [164]:
size_input = 784
size_hidden_1 = 256
size_hidden_2 = 128
size_output = 10

In [151]:
(X_train, y_train),(X_test, y_test) = mnist.load_data()
X_train = tf.cast(tf.reshape(X_train, (-1, 784)), dtype=tf.float32)
X_test = tf.cast(tf.reshape(X_test, (-1, 784)), dtype=tf.float32)
y_train = tf.one_hot(y_train, 10)
y_test = tf.one_hot(y_test, 10)
X_train, X_test = X_train / 255.0, X_test / 255.0

In [ ]:
print(X_train[0])
#print(y_train[0])
print(tf.shape(X_test))
print(tf.shape(y_test))

In [170]:
#split dataset into batches
train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train)).batch(30)
test_ds = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(100)

Build MLP using Eager Execution

In [187]:
class MLP(object):
  def __init__(self, size_input, size_hidden_1, size_hidden_2, size_output, device=None):
    """
    size_input: int, size of input layer
    size_hidden: int, size of hidden layer
    size_output: int, size of output layer
    device: etiher 'cpu' or 'gpu' or None. If None, decided automatically during eager.
    """
    self.size_input, self.size_hidden_1, self.size_hidden_2, self.size_output, self.device =\
    size_input, size_hidden_1, size_hidden_2, size_output, device

    #Initialize weights between input layer and hidden layer
    self.W1 = tf.Variable(tf.random.normal([self.size_input, self.size_hidden_1]))
    self.b1 = tf.Variable(tf.random.normal([1,self.size_hidden_1]))
    self.W2 = tf.Variable(tf.random.normal([self.size_hidden_1, self.size_hidden_2]))
    self.b2 = tf.Variable(tf.random.normal([1,self.size_hidden_2]))
    self.Wop = tf.Variable(tf.random.normal([self.size_hidden_2, self.size_output]))
    self.bop = tf.Variable(tf.random.normal([1,self.size_output]))

    self.variables = [self.W1, self.W2, self.b1, self.b2, self.Wop, self.bop]  # define variables(parameters) that will be updated 

  def forward(self, X): # (X is the input matrix)
    """
    forward pass
    X: Tensor, inputs
    """
    if self.device is not None:
      with tf.device('gpu:0' if self.device=='gpu' else 'cpu'):
        self.y=self.compute_output(X)
    else:
      self.y = self.compute_output(X)
      
    return self.y
    
  def loss(self, y_pred, y_true):
    '''
    y_pred - Tensor of shape(batch_size, size_output)
    y_true - Tensor of shape(batch_size, size_output)
    '''
    #y_true_tf = tf.cast(tf.reshape(y_true, (-1, self.size_output)), dtype=tf.float32)
    y_true_tf = tf.cast(y_true, dtype=tf.float32)
    y_pred_tf = tf.cast(y_pred, dtype=tf.float32)
    return tf.keras.losses.categorical_crossentropy(y_true, y_pred)

  def backward(self, X_train, y_train):
    '''
    backward pass
    '''
    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
    with tf.GradientTape() as tape:
      predicted = self.forward(X_train)
      current_loss = self.loss(predicted, y_train)
    grads = tape.gradient(current_loss, self.variables)
    optimizer.apply_gradients( zip( grads, self.variables))
    
  def compute_output(self, X):
#    X_tf = tf.cast(X, dytpe=tf.float32)
    what1 = tf.matmul(X, self.W1) + self.b1
    hhat1 = tf.nn.relu(what1)
    what2 = tf.matmul(hhat1, self.W2) + self.b2
    hhat2 = tf.nn.relu(what2)
    output = tf.nn.softmax(tf.matmul(hhat2, self.Wop) + self.bop)

    return output


Train Model

In [148]:
NUM_EPOCHS = 10

In [188]:
# Initialize model using CPU
mlp_on_gpu = MLP(size_input, size_hidden_1, size_hidden_2, size_output, device='gpu')

time_start = time.time()
for epoch in range(NUM_EPOCHS):
  loss_total = tf.zeros([1,1],dtype=tf.float32)
  lt = 0
  accuracy = []
  m = tf.keras.metrics.CategoricalAccuracy()
  train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(100, seed=epoch*(4208)).batch(100)
  for inputs, outputs in train_ds:
    preds = mlp_on_gpu.forward(inputs)
    loss_total = loss_total + mlp_on_gpu.loss(preds, outputs)
    lt = lt + mlp_on_gpu.loss(preds, outputs)
    m.update_state(preds,outputs)
    accuracy.append(m.result().numpy())
    mlp_on_gpu.backward(inputs, outputs)
  #print('Number of Epoch = {} - Average MSE:= {}'.format(epoch + 1, np.sum(loss_total) / 600))
  m.reset_state()
  print('Number of Epoch = {} - Accuracy:= {}'.format(epoch + 1, np.sum(accuracy)/len(accuracy)))
time_taken = time.time() - time_start
print('\nTotal time taken (in seconds): {:.2f}'.format(time_taken))

Number of Epoch = 1 - Accuracy:= 0.18871610005696615
Number of Epoch = 2 - Accuracy:= 0.30731590270996095
Number of Epoch = 3 - Accuracy:= 0.3425794219970703
Number of Epoch = 4 - Accuracy:= 0.37742151896158854
Number of Epoch = 5 - Accuracy:= 0.4426384989420573
Number of Epoch = 6 - Accuracy:= 0.476187998453776
Number of Epoch = 7 - Accuracy:= 0.49362213134765626
Number of Epoch = 8 - Accuracy:= 0.5032605997721354
Number of Epoch = 9 - Accuracy:= 0.5099298095703125
Number of Epoch = 10 - Accuracy:= 0.5243195597330729

Total time taken (in seconds): 387.46


In [186]:
test_loss_total = tf.Variable(0,dtype=tf.float32)
accuracy = []
m = tf.keras.metrics.CategoricalAccuracy()
for inputs, outputs in test_ds:
  preds = mlp_on_gpu.forward(inputs)
  test_loss_total = test_loss_total + mlp_on_gpu.loss(preds, outputs)
  m.update_state(preds,outputs)
  accuracy.append(m.result().numpy())
#print('Test MSE: {:.4f}'.format(np.sum(test_loss_total.numpy()) / X_train.shape[0]))
print('Accuracy: {:.4f}'.format(np.sum(accuracy) / len(accuracy)))

Accuracy: 0.8691
